In [1]:
import pickle
import os
from tqdm.notebook import tqdm,tnrange
from nltk.stem import PorterStemmer,SnowballStemmer,WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
import json

#punkt and stopqwords
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/darshparikh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/darshparikh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
os.chdir('../CSE508_Winter2023_Dataset/CSE508_Winter2023_Dataset_XML/')

In [3]:
def preprocess(filename):
    # open the file and read the text
    with open(filename, 'r') as f:
        text = f.read()
    # Lowercase the text
    text = text.lower()

    # Perform tokenization
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # Remove punctuations
    tokens = [w for w in tokens if w.isalpha()]
    # Remove blank space tokens
    tokens = [w for w in tokens if w.strip()]
    # Remove tokens that occur twice, preserve the order of tokens in list
    tokens = list(dict.fromkeys(tokens))
    return tokens

# make a dictionary of bigram tokens as keys containg the list of files they occur in
bigram_dict = {}
def make_dict(filename, tokens):
    for i in range(len(tokens)-1):
        bigram = tokens[i] + ' ' + tokens[i+1]
        if bigram in bigram_dict:
            bigram_dict[bigram].append(filename)
        else:
            bigram_dict[bigram] = [filename]

In [4]:
# run the function preprocess and make_dict for all the files in the directory
for filename in tqdm(os.listdir()):
    tokens = preprocess(filename)
    make_dict(filename, tokens)

# save the dictionary in a pickle file
os.chdir('../InvertedIndex')
# delete file with given filename in directory
if os.path.exists('bigramindex.pickle'):
    os.remove('bigramindex.pickle')

# save the dictionary in a pickle file
with open('bigramindex.pickle', 'wb') as f:
    pickle.dump(bigram_dict, f)

# delete file with given filename in directory
if os.path.exists('bigramindex.txt'):
    os.remove('bigramindex.txt')

with open('bigramindex.txt', 'w') as f:
    f.write(json.dumps(bigram_dict, indent=4))

In [5]:
os.chdir('../CSE508_Winter2023_Dataset_XML/')

In [6]:
db = {}

def make_dict(tokens, filename):
    for i in range(len(tokens)):
        if tokens[i] in db:
            db[tokens[i]].append([filename, i])
        else:
            db[tokens[i]] = [[filename, i]]


In [7]:
for filename in tqdm(os.listdir()):
    tokens = preprocess(filename)
    make_dict(tokens, filename)

# save the dictionary in a pickle file
os.chdir('../InvertedIndex')
# delete file with given filename in directory
if os.path.exists('positional.pickle'):
    os.remove('positional.pickle')

# save the dictionary in a pickle file
with open('positional.pickle', 'wb') as f:
    pickle.dump(db, f)

# delete file with given filename in directory
if os.path.exists('positional.txt'):
    os.remove('positional.txt')

with open('posotional.txt', 'w') as f:
    f.write(json.dumps(db, indent=4))